<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#1.-Import-Packages" data-toc-modified-id="1.-Import-Packages-1">1. Import Packages</a></span></li><li><span><a href="#2.-2017-2018-NBA-Game-Logs-Scraping" data-toc-modified-id="2.-2017-2018-NBA-Game-Logs-Scraping-2">2. 2017-2018 NBA Game Logs Scraping</a></span></li><li><span><a href="#3.-2016-2017-NBA-Game-Logs-Scraping" data-toc-modified-id="3.-2016-2017-NBA-Game-Logs-Scraping-3">3. 2016-2017 NBA Game Logs Scraping</a></span></li><li><span><a href="#4.-2015-2016-NBA-Game-Logs-Scraping" data-toc-modified-id="4.-2015-2016-NBA-Game-Logs-Scraping-4">4. 2015-2016 NBA Game Logs Scraping</a></span></li></ul></div>

# 1. Import Packages

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import regex as re
from bs4 import BeautifulSoup
import numpy as np
import time

# 2. 2017-2018 NBA Game Logs Scraping

In [2]:
# set webdriver for scraping and create the beginning link that we start from
driver = webdriver.Chrome('./chromedriver')

driver.get('https://cleaningtheglass.com/stats/players?season=2017')
time.sleep(4)

In [3]:
# input cleaning the glass user name and password
user_name = ''   # input Cleaning the Glass user name
password = ''    # input Cleaning the Glass password

In [4]:
# log in using our cleaning the glass log in info - login info will be removed at final submission
driver.find_element_by_id("click_to_login").click()
time.sleep(4)
driver.switch_to.frame(driver.find_element_by_tag_name('iframe'))
time.sleep(4)
driver.find_element_by_id('login').send_keys(user_name)
driver.find_element_by_id('password').send_keys(password)
driver.find_element_by_class_name('button').click()
time.sleep(4)
driver.switch_to.default_content()
time.sleep(4)

In [5]:
# find the table on the seasonal players data
table = driver.find_element_by_class_name('stat_table_container')
time.sleep(4)

In [6]:
season = table.text.split('\n')
# cleaning out column headers
# this is part of the process to create a player list for our scraping for loop
remove_str = ['Player',
 'Age',
 'Team',
 'Pos',
 'MIN',
 'Usage',
 'PSA',
 'AST%',
 'AST:Usg',
 'TOV%',
 '%',
 '#',
 '%',
 '#',
 '%',
 '#',
 '%',
 '#',
 '%',
 '#']

for char in remove_str:
    season.remove(char)

In [7]:
# creating players list to know how many iterations we need in our final scraping loop
# we note that this does result in duplicate players for ones who switch teams midseason
        # acceptable as this will be cleaned when we drop duplicate rows from our final frame
players = []
for row in season:
    new_list = row.split(' ')
    player = new_list[0] + ' ' + new_list[1]
    players.append(player)

In [8]:
# creating new column list to better order our columns
# this data comes from work completed after this stage to determine the best columns order but 
# frame needs to be instantiated before the for loop to create the base frame to add each player frame onto it
column_list = ['date', 'name',  'age', 'position', 'height', 'location',
               'opponent', 'result', 'games_to_date', 'days_off','team_pts', 'opp_pts', 'total_points',
               'mins_played', 'mins_played_to_date', 'avg_mins_played', 'pts_scored', 
               'rebounds', 'assists', 'fg_made_attempts', 'fg_attempts', 'fg_attempts_to_date',
               '3p_made_attempts', 'ft_made_attempts', 'ft_attempts',
               'ft_attempts_to_date', 'usage_percentile', 'usage',
               'psa_pct', 'psa', 'ast%_pct', 'ast%', 'tov%_pct', 'tov%', 'astd%_pct',
               'astd%', 'fgor%_pct', 'fgor%', 'fgdr%_pct', 'fgdr%', 'block%_pct',
               'block%', 'stl%_pct', 'stl%']

nba_player_data_2017_2018 = pd.DataFrame(columns = column_list)

In [9]:
driver.find_element_by_class_name('stat_table_container')

<selenium.webdriver.remote.webelement.WebElement (session="b0c42c18245314c5f2e5308be687fbd7", element="6e5fe30c-1f5a-42e7-84b7-7fd7476610f5")>

In [10]:
# creating the list of player strings to pass into the xpath, xpath requires this be string
player_list_string = [str(i) for i in list((range(1, len(players) + 1)))]

# loop to go through each player in the season data and capture their game logs
for player in player_list_string:
    player_x_path = '//*[@id="players-table"]/tbody/tr[' + player + ']/td[1]/a'
    driver.find_element_by_xpath(player_x_path).click()
    try:
        # obtaining player name
        # removing dots, dashes, and spaces as we want to standardize names for when we combine frames from 
        # different sources
        name = driver.find_element_by_xpath('//*[@id="player_name"]').text.lower().replace(
            ' ', '').replace(
            '.', '').replace(
            '-', '').replace(
            "'", "")
        # obtaining player height
        height = driver.find_element_by_xpath('//*[@id="player_info"]/span[1]').text
        # we want to convert this to inches to allow for input into model
        height_dimensions = height.replace("\'", " ").replace('"', '').split(' ')
        for dim in height_dimensions:
            feet_float = int(height_dimensions[0])
            inch_float = int(height_dimensions[1])
            height = 12*feet_float + inch_float
        # obtaining player position
        position = driver.find_element_by_xpath('//*[@id="player_info"]/span[2]').text.lower()
        # subtracting age by 2 for 2017-2018 data as age is not updated from the current 2019-2020 season
        # although age isn't dynamic, we are comfortable with this as variation should be picked up by date/games played
        age = float(driver.find_element_by_xpath('//*[@id="player_info"]/span[3]').text.replace(' YEARS OLD', '')) - 2

        # next we begin the process of obtaining the player game logs for the season

        driver.find_element_by_xpath('//*[@id="player_page_section_gamelogs"]').click()
        time.sleep(1)

        # we need try/except to account for possible missing data on webpages (rare but needs to be accounted for)
        # go back to 2018-2019 season
        try:
            driver.find_element_by_xpath('//*[@id="year_nav"]/a[1]').click()
            time.sleep(1)
            try:
                # go back to 2017-2018 season
                driver.find_element_by_xpath('//*[@id="year_nav"]/a[1]').click()
                time.sleep(1)

                try:
                    # accesses the game log table
                    table2 = driver.find_element_by_class_name('stat_table_container')
                    # accesses the content of the table (the table body)
                    body = table2.find_element_by_xpath('//*[@id="content_section_wrapper"]/div[2]/div[3]/div[1]/table/tbody')
                    # accesses each individual row (game) in the table
                    rows = body.find_elements_by_tag_name('tr')
                    # accessing each data cell in the table, creates a list for each row where elements of the list are each data
                    # point in the row
                    cells = [row.find_elements_by_tag_name('td') for row in rows]

                    # updating the data to get it into proper python/pandas format
                    game_set = [[point.text for point in game] for game in cells]

                    # creating lists of our data types to pass through into the data frame
                    dates = []
                    locs = []
                    opps = []
                    results = []
                    team_pts = []
                    opp_pts = []
                    minutes = []
                    pts = []
                    rebs = []
                    asts = []
                    fgm = []
                    threepm = []
                    ftm = []
                    usage_pct = []
                    usage = []
                    psa_pct = []
                    psa = []
                    ast_pct =[]
                    ast = []
                    tov_pct = []
                    tov = []
                    astd_pct = []
                    astd = []
                    fgor_pct = []
                    fgor = []
                    fgdr_pct = []
                    fgdr = []
                    blk_pct = []
                    blk = []
                    stl_pct = []
                    stl = []
                    for i in game_set:
                        dates.append(i[0])
                        locs.append(i[1])
                        opps.append(i[2])
                        results.append(i[3])
                        team_pts.append(i[4])
                        opp_pts.append(i[5])
                        minutes.append(i[6])
                        pts.append(i[7])
                        rebs.append(i[8])
                        asts.append(i[9])
                        fgm.append(i[10])
                        threepm.append(i[11])
                        ftm.append(i[12])
                        usage_pct.append(i[13])
                        usage.append(i[14])
                        psa_pct.append(i[15])
                        psa.append(i[16])
                        ast_pct.append(i[17])
                        ast.append(i[18])
                        tov_pct.append(i[19])
                        tov.append(i[20])
                        astd_pct.append(i[21])
                        astd.append(i[22])
                        fgor_pct.append(i[23])
                        fgor.append(i[24])
                        fgdr_pct.append(i[25])
                        fgdr.append(i[26])
                        blk_pct.append(i[27])
                        blk.append(i[28])
                        stl_pct.append(i[29])
                        stl.append(i[30])
                    # next we instantiate a dictionary to create our player data frame
                    player_season_data = {
                        'date' : dates,
                        'location' : locs,
                        'opponent' : opps,
                        'result' : results,
                        'team_pts' : team_pts,
                        'opp_pts' : opp_pts,
                        'mins_played' : minutes,
                        'pts_scored' : pts,
                        'rebounds' : rebs,
                        'assists' : asts, 
                        'fg_made_attempts' : fgm,
                        '3p_made_attempts' : threepm,
                        'ft_made_attempts' : ftm,
                        'usage_percentile' : usage_pct,
                        'usage' : usage,
                        'psa_pct' : psa_pct,
                        'psa' : psa,
                        'ast%_pct' : ast_pct,
                        'ast%' : ast,
                        'tov%_pct' : tov_pct,
                        'tov%' : tov,
                        'astd%_pct' : astd_pct,
                        'astd%' : astd,
                        'fgor%_pct' : fgor_pct,
                        'fgor%' : fgor,
                        'fgdr%_pct' : fgdr_pct,
                        'fgdr%' : fgdr,
                        'block%_pct' : blk_pct,
                        'block%' : blk,
                        'stl%_pct' : stl_pct,
                        'stl%' : stl  
                    }


                    # creating the individual player data frame
                    player_frame = pd.DataFrame(player_season_data)

                    # changing some columns to integers

                    int_cols = ['team_pts','opp_pts','mins_played','pts_scored','rebounds','assists']

                    player_frame[int_cols] = player_frame[int_cols].apply(pd.to_numeric)

                    # we need to add in our data to the frame that wasn't contained in the game logs table
                    player_frame['name'] = name
                    player_frame['age'] = age
                    player_frame['position'] = position
                    player_frame['height'] = height

                    # making changes to some columns for clarity
                    player_frame['location'] = player_frame['location'].map({'@' : 'road', 'vs.' : 'home'})

                    # creating new columns based on our existing versions

                    # running total of total minutes played to date
                    player_frame['mins_played_to_date'] = [sum(player_frame['mins_played'][(i+1):player_frame.shape[0]].values)
                                                           for i in range(player_frame.shape[0])]

                    # running average of minutes played per game
                    #we will include the current game in our running average to eliminate nans
                    player_frame['avg_mins_played'] = [(player_frame['mins_played'][(i):player_frame.shape[0]].values).mean() 
                                                       for i in range(player_frame.shape[0])]

                    # running total of total games played to date
                    player_frame['games_to_date'] = list(range(player_frame.shape[0]))[::-1]

                    # next we will unpack the field goal / free throw attempts data as they are currently string
                    # we only care about the attempts as a metric for workload 

                    player_frame['fg_attempts'] = [int(player_frame.loc[i, 'fg_made_attempts'].split('/')[1])
                                                   for i in range(player_frame.shape[0])]

                    # running total of field goal attempts
                    player_frame['fg_attempts_to_date'] = [sum(player_frame['fg_attempts'][(i+1):player_frame.shape[0]].values)
                                                           for i in range(player_frame.shape[0])]

                    # now the same for free throws.  this might be useful given their association with contact
                    player_frame['ft_attempts'] = [int(player_frame.loc[i, 'ft_made_attempts'].split('/')[1])
                                                   for i in range(player_frame.shape[0])]

                    # running total of free throw attempts
                    player_frame['ft_attempts_to_date'] = [sum(player_frame['ft_attempts'][(i+1):player_frame.shape[0]].values)
                                                           for i in range(player_frame.shape[0])]

                    # number of total points in game as a proxy for pace
                    player_frame['total_points'] = player_frame['team_pts'] + player_frame['opp_pts']

                    # handling dates
                    # we want to updated the dates to datetime object and also see if we can extract days off
                    # 1 day off if game was played next day, two if you had a full 48 hour break, etc.
                    player_frame['date'] = pd.to_datetime(player_frame['date'])
                    # next we can create a new column that shows the days between the last two games
                    # first game of season assumed to have 4 off days from end of preseason based on 2019 NBA calender 
                    # (used as proxy for other seasons as well for simplicity)
                    player_frame['days_off'] = [(player_frame.loc[i, 'date'] - player_frame.loc[(i+1), 'date']).days 
                                                if i < (player_frame.shape[0] - 1)
                                                else (pd.to_datetime('2019-10-22') - pd.to_datetime('2019-10-18')).days
                                                for i in range(player_frame.shape[0])]

                    # reindexing columns in our choice of format
                    player_frame = player_frame.reindex(columns = column_list)

                    # adding on individual player data frame to overall players data frame
                    nba_player_data_2017_2018 = pd.concat([nba_player_data_2017_2018, player_frame], axis = 0)

                    driver.back()
                    time.sleep(1)
                    driver.back()
                    time.sleep(1)
                    driver.back()
                    time.sleep(1)
                    driver.back()
                    time.sleep(1)

                except:
                    driver.back()
                    time.sleep(1)
                    driver.back()
                    time.sleep(1)
                    driver.back()
                    time.sleep(1)
                    driver.back()
                    time.sleep(1)
            except:
                driver.back()
                time.sleep(1)
                driver.back()
                time.sleep(1)
                driver.back()
                time.sleep(1)
        except:
            driver.back()
            time.sleep(1)
            driver.back()
            time.sleep(1)
    except:
        driver.back()
        time.sleep(1)

In [11]:
# saving data to csv
nba_player_data_2017_2018.to_csv('../data/2017_2018_game_logs.csv')

# 3. 2016-2017 NBA Game Logs Scraping

In [12]:
# set webdriver for scraping and create the beginning link that we start from
driver = webdriver.Chrome('./chromedriver')

driver.get('https://cleaningtheglass.com/stats/players?season=2016')
time.sleep(4)

In [13]:
# log in using our cleaning the glass log in info - login info will be removed at final submission
driver.find_element_by_id("click_to_login").click()
time.sleep(4)
driver.switch_to.frame(driver.find_element_by_tag_name('iframe'))
time.sleep(4)
driver.find_element_by_id('login').send_keys(user_name)
driver.find_element_by_id('password').send_keys(password)
driver.find_element_by_class_name('button').click()
time.sleep(4)
driver.switch_to.default_content()
time.sleep(4)

In [14]:
# find the table on the seasonal players data
table = driver.find_element_by_class_name('stat_table_container')
time.sleep(4)

In [15]:
season = table.text.split('\n')
# cleaning out column headers
# this is part of the process to create a player list for our scraping for loop
remove_str = ['Player',
 'Age',
 'Team',
 'Pos',
 'MIN',
 'Usage',
 'PSA',
 'AST%',
 'AST:Usg',
 'TOV%',
 '%',
 '#',
 '%',
 '#',
 '%',
 '#',
 '%',
 '#',
 '%',
 '#']

for char in remove_str:
    season.remove(char)

In [16]:
# creating players list to know how many iterations we need in our final scraping loop
# we note that this does result in duplicate players for ones who switch teams midseason
        # acceptable as this will be cleaned when we drop duplicate rows from our final frame
players = []
for row in season:
    new_list = row.split(' ')
    player = new_list[0] + ' ' + new_list[1]
    players.append(player)

In [17]:
# creating new column list to better order our columns
# this data comes from work completed after this stage to determine the best columns order but 
# frame needs to be instantiated before the for loop to create the base frame to add each player frame onto it
column_list = ['date', 'name',  'age', 'position', 'height', 'location',
               'opponent', 'result', 'games_to_date', 'days_off','team_pts', 'opp_pts', 'total_points',
               'mins_played', 'mins_played_to_date', 'avg_mins_played', 'pts_scored', 
               'rebounds', 'assists', 'fg_made_attempts', 'fg_attempts', 'fg_attempts_to_date',
               '3p_made_attempts', 'ft_made_attempts', 'ft_attempts',
               'ft_attempts_to_date', 'usage_percentile', 'usage',
               'psa_pct', 'psa', 'ast%_pct', 'ast%', 'tov%_pct', 'tov%', 'astd%_pct',
               'astd%', 'fgor%_pct', 'fgor%', 'fgdr%_pct', 'fgdr%', 'block%_pct',
               'block%', 'stl%_pct', 'stl%']

nba_player_data_2016_2017 = pd.DataFrame(columns = column_list)

In [18]:
driver.find_element_by_class_name('stat_table_container')

<selenium.webdriver.remote.webelement.WebElement (session="6fdfe70d9da83a0c89569aefdcf9533f", element="968d4ac7-03a5-4048-92fc-590018381db9")>

In [19]:
# creating the list of player strings to pass into the xpath, xpath requires this be string
player_list_string = [str(i) for i in list((range(1, len(players) + 1)))]

# loop to go through each player in the season data and capture their game logs
for player in player_list_string:
    player_x_path = '//*[@id="players-table"]/tbody/tr[' + player + ']/td[1]/a'
    driver.find_element_by_xpath(player_x_path).click()
    try:
        # obtaining player name
        # removing dots, dashes, and spaces as we want to standardize names for when we combine frames from 
        # different sources
        name = driver.find_element_by_xpath('//*[@id="player_name"]').text.lower().replace(
            ' ', '').replace(
            '.', '').replace(
            '-', '').replace(
            "'", "")
        # obtaining player height
        height = driver.find_element_by_xpath('//*[@id="player_info"]/span[1]').text
        # we want to convert this to inches to allow for input into model
        height_dimensions = height.replace("\'", " ").replace('"', '').split(' ')
        for dim in height_dimensions:
            feet_float = int(height_dimensions[0])
            inch_float = int(height_dimensions[1])
            height = 12*feet_float + inch_float
        # obtaining player position
        position = driver.find_element_by_xpath('//*[@id="player_info"]/span[2]').text.lower()
        # subtracting age by 3 for 2016-2017 data as age is not updated from the current 2019-2020 season
        # although age isn't dynamic, we are comfortable with this as variation should be picked up by date/games played
        age = float(driver.find_element_by_xpath('//*[@id="player_info"]/span[3]').text.replace(' YEARS OLD', '')) - 3

        # next we begin the process of obtaining the player game logs for the season

        driver.find_element_by_xpath('//*[@id="player_page_section_gamelogs"]').click()
        time.sleep(1)

        # we need try/except to account for possible missing data on webpages (rare but needs to be accounted for)
        # go back to 2018-2019 season
        try:
            driver.find_element_by_xpath('//*[@id="year_nav"]/a[1]').click()
            time.sleep(1)
            try:
                driver.find_element_by_xpath('//*[@id="year_nav"]/a[1]').click()
                time.sleep(1)
                try:
                    # go back to 2017-2018 season
                    driver.find_element_by_xpath('//*[@id="year_nav"]/a[1]').click()
                    time.sleep(1)

                    try:
                        # accesses the game log table
                        table2 = driver.find_element_by_class_name('stat_table_container')
                        # accesses the content of the table (the table body)
                        body = table2.find_element_by_xpath('//*[@id="content_section_wrapper"]/div[2]/div[3]/div[1]/table/tbody')
                        # accesses each individual row (game) in the table
                        rows = body.find_elements_by_tag_name('tr')
                        # accessing each data cell in the table, creates a list for each row where elements of the list are each data
                        # point in the row
                        cells = [row.find_elements_by_tag_name('td') for row in rows]

                        # updating the data to get it into proper python/pandas format
                        game_set = [[point.text for point in game] for game in cells]

                        # creating lists of our data types to pass through into the data frame
                        dates = []
                        locs = []
                        opps = []
                        results = []
                        team_pts = []
                        opp_pts = []
                        minutes = []
                        pts = []
                        rebs = []
                        asts = []
                        fgm = []
                        threepm = []
                        ftm = []
                        usage_pct = []
                        usage = []
                        psa_pct = []
                        psa = []
                        ast_pct =[]
                        ast = []
                        tov_pct = []
                        tov = []
                        astd_pct = []
                        astd = []
                        fgor_pct = []
                        fgor = []
                        fgdr_pct = []
                        fgdr = []
                        blk_pct = []
                        blk = []
                        stl_pct = []
                        stl = []
                        for i in game_set:
                            dates.append(i[0])
                            locs.append(i[1])
                            opps.append(i[2])
                            results.append(i[3])
                            team_pts.append(i[4])
                            opp_pts.append(i[5])
                            minutes.append(i[6])
                            pts.append(i[7])
                            rebs.append(i[8])
                            asts.append(i[9])
                            fgm.append(i[10])
                            threepm.append(i[11])
                            ftm.append(i[12])
                            usage_pct.append(i[13])
                            usage.append(i[14])
                            psa_pct.append(i[15])
                            psa.append(i[16])
                            ast_pct.append(i[17])
                            ast.append(i[18])
                            tov_pct.append(i[19])
                            tov.append(i[20])
                            astd_pct.append(i[21])
                            astd.append(i[22])
                            fgor_pct.append(i[23])
                            fgor.append(i[24])
                            fgdr_pct.append(i[25])
                            fgdr.append(i[26])
                            blk_pct.append(i[27])
                            blk.append(i[28])
                            stl_pct.append(i[29])
                            stl.append(i[30])
                        # next we instantiate a dictionary to create our player data frame
                        player_season_data = {
                            'date' : dates,
                            'location' : locs,
                            'opponent' : opps,
                            'result' : results,
                            'team_pts' : team_pts,
                            'opp_pts' : opp_pts,
                            'mins_played' : minutes,
                            'pts_scored' : pts,
                            'rebounds' : rebs,
                            'assists' : asts, 
                            'fg_made_attempts' : fgm,
                            '3p_made_attempts' : threepm,
                            'ft_made_attempts' : ftm,
                            'usage_percentile' : usage_pct,
                            'usage' : usage,
                            'psa_pct' : psa_pct,
                            'psa' : psa,
                            'ast%_pct' : ast_pct,
                            'ast%' : ast,
                            'tov%_pct' : tov_pct,
                            'tov%' : tov,
                            'astd%_pct' : astd_pct,
                            'astd%' : astd,
                            'fgor%_pct' : fgor_pct,
                            'fgor%' : fgor,
                            'fgdr%_pct' : fgdr_pct,
                            'fgdr%' : fgdr,
                            'block%_pct' : blk_pct,
                            'block%' : blk,
                            'stl%_pct' : stl_pct,
                            'stl%' : stl  
                        }


                        # creating the individual player data frame
                        player_frame = pd.DataFrame(player_season_data)

                        # changing some columns to integers

                        int_cols = ['team_pts','opp_pts','mins_played','pts_scored','rebounds','assists']

                        player_frame[int_cols] = player_frame[int_cols].apply(pd.to_numeric)

                        # we need to add in our data to the frame that wasn't contained in the game logs table
                        player_frame['name'] = name
                        player_frame['age'] = age
                        player_frame['position'] = position
                        player_frame['height'] = height

                        # making changes to some columns for clarity
                        player_frame['location'] = player_frame['location'].map({'@' : 'road', 'vs.' : 'home'})

                        # creating new columns based on our existing versions

                        # running total of total minutes played to date
                        player_frame['mins_played_to_date'] = [sum(player_frame['mins_played'][(i+1):player_frame.shape[0]].values)
                                                               for i in range(player_frame.shape[0])]

                        # running average of minutes played per game
                        #we will include the current game in our running average to eliminate nans
                        player_frame['avg_mins_played'] = [(player_frame['mins_played'][(i):player_frame.shape[0]].values).mean() 
                                                           for i in range(player_frame.shape[0])]

                        # running total of total games played to date
                        player_frame['games_to_date'] = list(range(player_frame.shape[0]))[::-1]

                        # next we will unpack the field goal / free throw attempts data as they are currently string
                        # we only care about the attempts as a metric for workload 

                        player_frame['fg_attempts'] = [int(player_frame.loc[i, 'fg_made_attempts'].split('/')[1])
                                                       for i in range(player_frame.shape[0])]

                        # running total of field goal attempts
                        player_frame['fg_attempts_to_date'] = [sum(player_frame['fg_attempts'][(i+1):player_frame.shape[0]].values)
                                                               for i in range(player_frame.shape[0])]

                        # now the same for free throws.  this might be useful given their association with contact
                        player_frame['ft_attempts'] = [int(player_frame.loc[i, 'ft_made_attempts'].split('/')[1])
                                                       for i in range(player_frame.shape[0])]

                        # running total of free throw attempts
                        player_frame['ft_attempts_to_date'] = [sum(player_frame['ft_attempts'][(i+1):player_frame.shape[0]].values)
                                                               for i in range(player_frame.shape[0])]

                        # number of total points in game as a proxy for pace
                        player_frame['total_points'] = player_frame['team_pts'] + player_frame['opp_pts']

                        # handling dates
                        # we want to updated the dates to datetime object and also see if we can extract days off
                        # 1 day off if game was played next day, two if you had a full 48 hour break, etc.
                        player_frame['date'] = pd.to_datetime(player_frame['date'])
                        # next we can create a new column that shows the days between the last two games
                        # first game of season assumed to have 4 off days from end of preseason based on 2019 NBA calender 
                        # (used as proxy for other seasons as well for simplicity)
                        player_frame['days_off'] = [(player_frame.loc[i, 'date'] - player_frame.loc[(i+1), 'date']).days 
                                                    if i < (player_frame.shape[0] - 1)
                                                    else (pd.to_datetime('2019-10-22') - pd.to_datetime('2019-10-18')).days
                                                    for i in range(player_frame.shape[0])]

                        # reindexing columns in our choice of format
                        player_frame = player_frame.reindex(columns = column_list)

                        # adding on individual player data frame to overall players data frame
                        nba_player_data_2016_2017 = pd.concat([nba_player_data_2016_2017, player_frame], axis = 0)

                        driver.back()
                        time.sleep(1)
                        driver.back()
                        time.sleep(1)
                        driver.back()
                        time.sleep(1)
                        driver.back()
                        time.sleep(1)
                        driver.back()
                        time.sleep(1)
                    except:
                        driver.back()
                        time.sleep(1)
                        driver.back()
                        time.sleep(1)
                        driver.back()
                        time.sleep(1)
                        driver.back()
                        time.sleep(1)
                        driver.back()
                        time.sleep(1)
                except:
                    driver.back()
                    time.sleep(1)
                    driver.back()
                    time.sleep(1)
                    driver.back()
                    time.sleep(1)
                    driver.back()
                    time.sleep(1)
            except:
                driver.back()
                time.sleep(1)
                driver.back()
                time.sleep(1)
                driver.back()
                time.sleep(1)
        except:
            driver.back()
            time.sleep(1)
            driver.back()
            time.sleep(1)
    except:
        driver.back()
        time.sleep(1)

In [20]:
# saving data to csv
nba_player_data_2016_2017.to_csv('../data/2016_2017_game_logs.csv')

# 4. 2015-2016 NBA Game Logs Scraping

In [21]:
# set webdriver for scraping and create the beginning link that we start from
driver = webdriver.Chrome('./chromedriver')

driver.get('https://cleaningtheglass.com/stats/players?season=2015')
time.sleep(4)

In [22]:
# log in using our cleaning the glass log in info - login info will be removed at final submission
driver.find_element_by_id("click_to_login").click()
time.sleep(4)
driver.switch_to.frame(driver.find_element_by_tag_name('iframe'))
time.sleep(4)
driver.find_element_by_id('login').send_keys(user_name)
driver.find_element_by_id('password').send_keys(password)
driver.find_element_by_class_name('button').click()
time.sleep(4)
driver.switch_to.default_content()
time.sleep(4)

In [23]:
# find the table on the seasonal players data
table = driver.find_element_by_class_name('stat_table_container')
time.sleep(4)

In [24]:
season = table.text.split('\n')
# cleaning out column headers
# this is part of the process to create a player list for our scraping for loop
remove_str = ['Player',
 'Age',
 'Team',
 'Pos',
 'MIN',
 'Usage',
 'PSA',
 'AST%',
 'AST:Usg',
 'TOV%',
 '%',
 '#',
 '%',
 '#',
 '%',
 '#',
 '%',
 '#',
 '%',
 '#']

for char in remove_str:
    season.remove(char)

In [25]:
# creating players list to know how many iterations we need in our final scraping loop
# we note that this does result in duplicate players for ones who switch teams midseason
        # acceptable as this will be cleaned when we drop duplicate rows from our final frame
players = []
for row in season:
    new_list = row.split(' ')
    player = new_list[0] + ' ' + new_list[1]
    players.append(player)

In [26]:
# creating new column list to better order our columns
# this data comes from work completed after this stage to determine the best columns order but 
# frame needs to be instantiated before the for loop to create the base frame to add each player frame onto it
column_list = ['date', 'name',  'age', 'position', 'height', 'location',
               'opponent', 'result', 'games_to_date', 'days_off','team_pts', 'opp_pts', 'total_points',
               'mins_played', 'mins_played_to_date', 'avg_mins_played', 'pts_scored', 
               'rebounds', 'assists', 'fg_made_attempts', 'fg_attempts', 'fg_attempts_to_date',
               '3p_made_attempts', 'ft_made_attempts', 'ft_attempts',
               'ft_attempts_to_date', 'usage_percentile', 'usage',
               'psa_pct', 'psa', 'ast%_pct', 'ast%', 'tov%_pct', 'tov%', 'astd%_pct',
               'astd%', 'fgor%_pct', 'fgor%', 'fgdr%_pct', 'fgdr%', 'block%_pct',
               'block%', 'stl%_pct', 'stl%']

nba_player_data_2015_2016 = pd.DataFrame(columns = column_list)

In [27]:
driver.find_element_by_class_name('stat_table_container')

<selenium.webdriver.remote.webelement.WebElement (session="e955bb8048ffae60cfcfcfacf65d5fde", element="ad1aa87b-1ca1-4e63-a7d1-0e4752ab0dc1")>

In [28]:
# creating the list of player strings to pass into the xpath, xpath requires this be string
player_list_string = [str(i) for i in list((range(1, len(players) + 1)))]

# loop to go through each player in the season data and capture their game logs
for player in player_list_string:
    player_x_path = '//*[@id="players-table"]/tbody/tr[' + player + ']/td[1]/a'
    driver.find_element_by_xpath(player_x_path).click()
    try:
        # obtaining player name
        # removing dots, dashes, and spaces as we want to standardize names for when we combine frames from 
        # different sources
        name = driver.find_element_by_xpath('//*[@id="player_name"]').text.lower().replace(
            ' ', '').replace(
            '.', '').replace(
            '-', '').replace(
            "'", "")
        # obtaining player height
        height = driver.find_element_by_xpath('//*[@id="player_info"]/span[1]').text
        # we want to convert this to inches to allow for input into model
        height_dimensions = height.replace("\'", " ").replace('"', '').split(' ')
        for dim in height_dimensions:
            feet_float = int(height_dimensions[0])
            inch_float = int(height_dimensions[1])
            height = 12*feet_float + inch_float
        # obtaining player position
        position = driver.find_element_by_xpath('//*[@id="player_info"]/span[2]').text.lower()
        # subtracting age by 4 for 2015-2016 data as age is not updated from the current 2019-2020 season
        # although age isn't dynamic, we are comfortable with this as variation should be picked up by date/games played
        age = float(driver.find_element_by_xpath('//*[@id="player_info"]/span[3]').text.replace(' YEARS OLD', '')) - 4

        # next we begin the process of obtaining the player game logs for the season

        driver.find_element_by_xpath('//*[@id="player_page_section_gamelogs"]').click()
        time.sleep(1)

        # we need try/except to account for possible missing data on webpages (rare but needs to be accounted for)
        # go back to 2018-2019 season
        try:
            driver.find_element_by_xpath('//*[@id="year_nav"]/a[1]').click()
            time.sleep(1)
            try:
                driver.find_element_by_xpath('//*[@id="year_nav"]/a[1]').click()
                time.sleep(1)
                try:
                    driver.find_element_by_xpath('//*[@id="year_nav"]/a[1]').click()
                    time.sleep(1)
                    try:
                        # go back to 2017-2018 season
                        driver.find_element_by_xpath('//*[@id="year_nav"]/a[1]').click()
                        time.sleep(1)

                        try:
                            # accesses the game log table
                            table2 = driver.find_element_by_class_name('stat_table_container')
                            # accesses the content of the table (the table body)
                            body = table2.find_element_by_xpath('//*[@id="content_section_wrapper"]/div[2]/div[3]/div[1]/table/tbody')
                            # accesses each individual row (game) in the table
                            rows = body.find_elements_by_tag_name('tr')
                            # accessing each data cell in the table, creates a list for each row where elements of the list are each data
                            # point in the row
                            cells = [row.find_elements_by_tag_name('td') for row in rows]

                            # updating the data to get it into proper python/pandas format
                            game_set = [[point.text for point in game] for game in cells]

                            # creating lists of our data types to pass through into the data frame
                            dates = []
                            locs = []
                            opps = []
                            results = []
                            team_pts = []
                            opp_pts = []
                            minutes = []
                            pts = []
                            rebs = []
                            asts = []
                            fgm = []
                            threepm = []
                            ftm = []
                            usage_pct = []
                            usage = []
                            psa_pct = []
                            psa = []
                            ast_pct =[]
                            ast = []
                            tov_pct = []
                            tov = []
                            astd_pct = []
                            astd = []
                            fgor_pct = []
                            fgor = []
                            fgdr_pct = []
                            fgdr = []
                            blk_pct = []
                            blk = []
                            stl_pct = []
                            stl = []
                            for i in game_set:
                                dates.append(i[0])
                                locs.append(i[1])
                                opps.append(i[2])
                                results.append(i[3])
                                team_pts.append(i[4])
                                opp_pts.append(i[5])
                                minutes.append(i[6])
                                pts.append(i[7])
                                rebs.append(i[8])
                                asts.append(i[9])
                                fgm.append(i[10])
                                threepm.append(i[11])
                                ftm.append(i[12])
                                usage_pct.append(i[13])
                                usage.append(i[14])
                                psa_pct.append(i[15])
                                psa.append(i[16])
                                ast_pct.append(i[17])
                                ast.append(i[18])
                                tov_pct.append(i[19])
                                tov.append(i[20])
                                astd_pct.append(i[21])
                                astd.append(i[22])
                                fgor_pct.append(i[23])
                                fgor.append(i[24])
                                fgdr_pct.append(i[25])
                                fgdr.append(i[26])
                                blk_pct.append(i[27])
                                blk.append(i[28])
                                stl_pct.append(i[29])
                                stl.append(i[30])
                            # next we instantiate a dictionary to create our player data frame
                            player_season_data = {
                                'date' : dates,
                                'location' : locs,
                                'opponent' : opps,
                                'result' : results,
                                'team_pts' : team_pts,
                                'opp_pts' : opp_pts,
                                'mins_played' : minutes,
                                'pts_scored' : pts,
                                'rebounds' : rebs,
                                'assists' : asts, 
                                'fg_made_attempts' : fgm,
                                '3p_made_attempts' : threepm,
                                'ft_made_attempts' : ftm,
                                'usage_percentile' : usage_pct,
                                'usage' : usage,
                                'psa_pct' : psa_pct,
                                'psa' : psa,
                                'ast%_pct' : ast_pct,
                                'ast%' : ast,
                                'tov%_pct' : tov_pct,
                                'tov%' : tov,
                                'astd%_pct' : astd_pct,
                                'astd%' : astd,
                                'fgor%_pct' : fgor_pct,
                                'fgor%' : fgor,
                                'fgdr%_pct' : fgdr_pct,
                                'fgdr%' : fgdr,
                                'block%_pct' : blk_pct,
                                'block%' : blk,
                                'stl%_pct' : stl_pct,
                                'stl%' : stl  
                            }


                            # creating the individual player data frame
                            player_frame = pd.DataFrame(player_season_data)

                            # changing some columns to integers

                            int_cols = ['team_pts','opp_pts','mins_played','pts_scored','rebounds','assists']

                            player_frame[int_cols] = player_frame[int_cols].apply(pd.to_numeric)

                            # we need to add in our data to the frame that wasn't contained in the game logs table
                            player_frame['name'] = name
                            player_frame['age'] = age
                            player_frame['position'] = position
                            player_frame['height'] = height

                            # making changes to some columns for clarity
                            player_frame['location'] = player_frame['location'].map({'@' : 'road', 'vs.' : 'home'})

                            # creating new columns based on our existing versions

                            # running total of total minutes played to date
                            player_frame['mins_played_to_date'] = [sum(player_frame['mins_played'][(i+1):player_frame.shape[0]].values)
                                                                   for i in range(player_frame.shape[0])]

                            # running average of minutes played per game
                            #we will include the current game in our running average to eliminate nans
                            player_frame['avg_mins_played'] = [(player_frame['mins_played'][(i):player_frame.shape[0]].values).mean() 
                                                               for i in range(player_frame.shape[0])]

                            # running total of total games played to date
                            player_frame['games_to_date'] = list(range(player_frame.shape[0]))[::-1]

                            # next we will unpack the field goal / free throw attempts data as they are currently string
                            # we only care about the attempts as a metric for workload 

                            player_frame['fg_attempts'] = [int(player_frame.loc[i, 'fg_made_attempts'].split('/')[1])
                                                           for i in range(player_frame.shape[0])]

                            # running total of field goal attempts
                            player_frame['fg_attempts_to_date'] = [sum(player_frame['fg_attempts'][(i+1):player_frame.shape[0]].values)
                                                                   for i in range(player_frame.shape[0])]

                            # now the same for free throws.  this might be useful given their association with contact
                            player_frame['ft_attempts'] = [int(player_frame.loc[i, 'ft_made_attempts'].split('/')[1])
                                                           for i in range(player_frame.shape[0])]

                            # running total of free throw attempts
                            player_frame['ft_attempts_to_date'] = [sum(player_frame['ft_attempts'][(i+1):player_frame.shape[0]].values)
                                                                   for i in range(player_frame.shape[0])]

                            # number of total points in game as a proxy for pace
                            player_frame['total_points'] = player_frame['team_pts'] + player_frame['opp_pts']

                            # handling dates
                            # we want to updated the dates to datetime object and also see if we can extract days off
                            # 1 day off if game was played next day, two if you had a full 48 hour break, etc.
                            player_frame['date'] = pd.to_datetime(player_frame['date'])
                            # next we can create a new column that shows the days between the last two games
                            # first game of season assumed to have 4 off days from end of preseason based on 2019 NBA calender 
                            # (used as proxy for other seasons as well for simplicity)
                            player_frame['days_off'] = [(player_frame.loc[i, 'date'] - player_frame.loc[(i+1), 'date']).days 
                                                        if i < (player_frame.shape[0] - 1)
                                                        else (pd.to_datetime('2019-10-22') - pd.to_datetime('2019-10-18')).days
                                                        for i in range(player_frame.shape[0])]

                            # reindexing columns in our choice of format
                            player_frame = player_frame.reindex(columns = column_list)

                            # adding on individual player data frame to overall players data frame
                            nba_player_data_2015_2016 = pd.concat([nba_player_data_2015_2016, player_frame], axis = 0)

                            driver.back()
                            time.sleep(1)
                            driver.back()
                            time.sleep(1)
                            driver.back()
                            time.sleep(1)
                            driver.back()
                            time.sleep(1)
                            driver.back()
                            time.sleep(1)
                            driver.back()
                            time.sleep(1)
                        except:
                            driver.back()
                            time.sleep(1)
                            driver.back()
                            time.sleep(1)
                            driver.back()
                            time.sleep(1)
                            driver.back()
                            time.sleep(1)
                            driver.back()
                            time.sleep(1)
                            driver.back()
                            time.sleep(1)
                    except:
                        driver.back()
                        time.sleep(1)
                        driver.back()
                        time.sleep(1)
                        driver.back()
                        time.sleep(1)
                        driver.back()
                        time.sleep(1)
                        driver.back()
                        time.sleep(1)
                except:
                    driver.back()
                    time.sleep(1)
                    driver.back()
                    time.sleep(1)
                    driver.back()
                    time.sleep(1)
                    driver.back()
                    time.sleep(1)
            except:
                driver.back()
                time.sleep(1)
                driver.back()
                time.sleep(1)
                driver.back()
                time.sleep(1)
        except:
            driver.back()
            time.sleep(1)
            driver.back()
            time.sleep(1)
    except:
        driver.back()
        time.sleep(1)

In [29]:
# saving data to csv
nba_player_data_2015_2016.to_csv('../data/2015_2016_game_logs.csv')

In [30]:
nba_player_data_2015_2016.shape

(23171, 44)

In [31]:
nba_player_data_2016_2017.shape

(23768, 44)

In [32]:
nba_player_data_2017_2018.shape

(24970, 44)